### Necessary Imports

In [1]:
# !pip install wikipedia

In [1]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from transformers import LlamaForCausalLM, LlamaTokenizer, TextStreamer
import huggingface_hub as hf_hub
import langchain
from langchain.llms.base import LLM
from langchain import PromptTemplate, LLMChain
import torch
from langchain.llms import OpenAI
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import AgentType


### Using Open AI LLM

In [5]:
## prompt
prompt_template = "Below is an instruction that describes a task. If question is related to pytorch Write a response that appropriately completes the request.\n\n### Instruction:\n{question}\n\n### Response:\n"

prompt = PromptTemplate(
            template=prompt_template, input_variables=["question"]
        )

## tool 1
wikipedia = WikipediaAPIWrapper()

## tool 2
openai_api_key = ''

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [6]:
tools = [
    Tool(
        name="pytorch search",
        func=llm_chain.run,
        description="Use this to answer questions only related to pytorch",
    ),
    Tool(
        name="wikipedia search",
        func=wikipedia.run,
        description="Use this to search wikipedia for general questions which is not related to pytorch",
    )
]

   
agent_chain = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)


In [7]:
# Do this so we can see exactly what's going on under the hood
langchain.debug = False

### Wiki Search Question

In [8]:
agent_chain.run('who is jeff bezos?')

Error in on_chain_start callback: 'name'


 I need to find out who Jeff Bezos is
Action: wikipedia search
Action Input: Jeff Bezos
Observation: Page: Jeff Bezos
Summary: Jeffrey Preston Bezos ( BAY-zohss; né Jorgensen; born January 12, 1964) is an American entrepreneur, media proprietor, investor, and commercial astronaut. He is the founder, executive chairman, and former president and CEO of Amazon, the world's largest e-commerce and cloud computing company. With a net worth of US$125 billion as of April 2023, Bezos is the third-wealthiest person in the world and was the wealthiest from 2017 to 2021, according to both the Bloomberg Billionaires Index and Forbes.Born in Albuquerque and raised in Houston and Miami, Bezos graduated from Princeton University in 1986. He holds a degree in electrical engineering and computer science. He worked on Wall Street in a variety of related fields from 1986 to early 1994. Bezos founded Amazon in late 1994 on a road trip from New York City to Seattle. The company began as an online bookstore 

"Jeff Bezos is an American entrepreneur, media proprietor, investor, and commercial astronaut. He is the founder, executive chairman, and former president and CEO of Amazon, the world's largest e-commerce and cloud computing company. He has a net worth of US$125 billion as of April 2023 and is the third-wealthiest person in the world. He is the half-brother of Mark Bezos, a space tourist and former advertising executive."

### PyTorch related question

In [9]:
agent_chain.run('Whats the difference between reshape and view in pytorch?')

Error in on_chain_start callback: 'name'


 I should look for an answer related to pytorch
Action: pytorch search
Action Input: reshape vs view
Observation: In PyTorch, the `reshape` and `view` functions are both used to change the shape of a tensor. The main difference between them is that `reshape` will create a new tensor, while `view` will modify the existing tensor. Additionally, `reshape` can change the number of elements in the tensor, while `view` cannot.
Thought: I now know the final answer
Final Answer: The main difference between reshape and view in pytorch is that reshape will create a new tensor, while view will modify the existing tensor. Additionally, reshape can change the number of elements in the tensor, while view cannot.

> Finished chain.


'The main difference between reshape and view in pytorch is that reshape will create a new tensor, while view will modify the existing tensor. Additionally, reshape can change the number of elements in the tensor, while view cannot.'

### Using Custom LLM

In [2]:
huggingface_api_key = ''

hf_hub.login(token=huggingface_api_key)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [3]:
model_name = 'shrinath-suresh/alpaca-lora-13b-answer-summary'

model = LlamaForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
    )

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
tokenizer = LlamaTokenizer.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [7]:
prompt_template = "Below is an instruction that describes a task. If question is related to pytorch Write a response that appropriately completes the request.\n\n### Instruction:\n{question}\n\n### Response:\n"

prompt = PromptTemplate(
            template=prompt_template, input_variables=["question"]
        )

## tool 1
wikipedia = WikipediaAPIWrapper()

In [8]:
class CustomLLM(LLM):
    def _call(self, prompt, stop=None) -> str:
        inputs = tokenizer([prompt], return_tensors="pt")

        response = model.generate(**inputs, max_new_tokens=256)
        response = tokenizer.decode(response[0])
        return response

    @property
    def _identifying_params(self):
        return {"name_of_model": model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

llm = CustomLLM()

In [9]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [16]:
tools = [
    Tool(
        name="pytorch search",
        func=llm_chain.run,
        description="Use this to answer questions only related to pytorch",
        return_direct=True
    ),
    Tool(
        name="wikipedia search",
        func=wikipedia.run,
        description="Use this to search wikipedia for general questions which is not related to pytorch",
        return_direct=True
    )
]

agent_chain = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)


In [20]:
# Set up the base template
agent_chain.agent.llm_chain.prompt.template = """Answer the following questions as best you can, but speaking as a pytorch engineer. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch
wikipedia search: Use this to search wikipedia for general questions which is not related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search, wikipedia search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pytorch engineer when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""


In [21]:
agent_chain.run("what is pytorch")

Error in on_chain_start callback: 'name'
/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Parsing LLM output produced both a final answer and a parse-able action:  ⁇  Answer the following questions as best you can, but speaking as a pytorch engineer. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch
wikipedia search: Use this to search wikipedia for general questions which is not related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search, wikipedia search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pytorch engineer when giving your final answer. Use lots of "Arg"s

Question: what is pytorch
Thought: pytorch is a python library for deep learning.

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.58 GiB (GPU 0; 22.19 GiB total capacity; 19.92 GiB already allocated; 714.50 MiB free; 20.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
agent_chain.run('who is jeff bezos?')

In [17]:
agent_chain.run('Whats the difference between reshape and view in pytorch?')

Error in on_chain_start callback: 'name'


Parsing LLM output produced both a final answer and a parse-able action:  ⁇  Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch
wikipedia search: Use this to search wikipedia for general questions which is not related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search, wikipedia search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Whats the difference between reshape and view in pytorch?
Thought: reshape is used to change the shape of a tensor while view is used to create a new tensor with the same shape as the original tens

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.21 GiB (GPU 0; 22.19 GiB total capacity; 15.02 GiB already allocated; 2.29 GiB free; 18.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF